# FINAL CODE

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
#Login to Google Drive and create drive object
g_login = GoogleAuth()
g_login.LocalWebserverAuth()
drive = GoogleDrive(g_login)
# Importing os and glob to find all PDFs inside subfolder

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=578078806716-7cijd18jhb0cp2hghbr1f3cfe56cdvdi.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



In [1]:
import numpy as np
import tensorflow as tf
import cv2
import time
import glob, os
from notify_run import Notify
notify = Notify()
from pydrive.drive import GoogleDrive
import serial #for Serial communication
import time   #for delay functions
#initializing Timing and mode

print('Initializing Arduino')
arduino = serial.Serial(port='/dev/ttyACM1',baudrate = 9600)   #Create Serial port object called arduinoSerialData
print(arduino.readline())
print('Initialized Arduino')


Mode=2
intruder=-2
    
# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
# org 
org = (0, 185) 
# fontScale 
fontScale = 1
# Red color in BGR 
color = (0, 0, 255) 
# Line thickness of 2 px 
thickness = 2
# Using cv2.putText() method 


def set_res(cap, x,y):
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, int(x))
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, int(y))
    return str(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),str(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.Session(graph=self.detection_graph)

        # Definite input and output Tensors for detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def processFrame(self, image):
        # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Actual detection.
        start_time = time.time()
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
        end_time = time.time()

        print("Elapsed Time:", end_time-start_time)

        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

    def close(self):
        self.sess.close()
        self.default_graph.close()
        
if __name__=='__main__':
    model_path = 'frozen_inference_graph.pb'
    odapi = DetectorAPI(path_to_ckpt=model_path)
    threshold = 0.4
    cap = cv2.VideoCapture("driving.mp4")
    ## Saving recorded Video
    ## 

    while True:
        r, img = cap.read()
        if r==True:
            boxes, scores, classes, num = odapi.processFrame(img)
            counter=0
            # Visualization of the results of a detection.
            for i in range(len(boxes)):
                # Class 1 represents human
                if classes[i] == 3 and scores[i] > threshold:
                    box = boxes[i]
                    cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,0),2) 
                    counter+=1
            if counter>10:
                img = cv2.putText(img,("Cars Detected: "+str(counter)+" Heavy Traffic This Way!"), org, font, fontScale,color, thickness, cv2.LINE_AA, False)
                arduino.write(str.encode('1'))
            else:
                img = cv2.putText(img,("Cars Detected: "+str(counter)+" Low Traffic This Way!"), org, font, fontScale,color, thickness, cv2.LINE_AA, False)
                arduino.write(str.encode('0'))
            cv2.imshow("Video Footage", img)
            key = cv2.waitKey(1)
            if key &0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

Initializing Arduino
b'Arduino Strarted \n'
Initialized Arduino
Elapsed Time: 3.6335747241973877
Elapsed Time: 0.21194100379943848
Elapsed Time: 0.16461658477783203
Elapsed Time: 0.132033109664917
Elapsed Time: 0.10640287399291992
Elapsed Time: 0.11621260643005371
Elapsed Time: 0.11417245864868164
Elapsed Time: 0.13856124877929688
Elapsed Time: 0.109588623046875
Elapsed Time: 0.13126158714294434
Elapsed Time: 0.10485219955444336
Elapsed Time: 0.12879061698913574
Elapsed Time: 0.10371136665344238
Elapsed Time: 0.11847257614135742
Elapsed Time: 0.10932397842407227
Elapsed Time: 0.13079166412353516
Elapsed Time: 0.11348772048950195
Elapsed Time: 0.12656450271606445
Elapsed Time: 0.1055755615234375
Elapsed Time: 0.118316650390625
Elapsed Time: 0.11702847480773926
Elapsed Time: 0.12041234970092773
Elapsed Time: 0.10268664360046387
Elapsed Time: 0.11580348014831543
Elapsed Time: 0.108123779296875
Elapsed Time: 0.11533117294311523
Elapsed Time: 0.1034846305847168
Elapsed Time: 0.1072633266448

Elapsed Time: 0.11037349700927734
Elapsed Time: 0.10100293159484863
Elapsed Time: 0.11595845222473145
Elapsed Time: 0.10297179222106934
Elapsed Time: 0.10998892784118652
Elapsed Time: 0.1019139289855957
Elapsed Time: 0.1109166145324707
Elapsed Time: 0.09611248970031738
Elapsed Time: 0.10959649085998535
Elapsed Time: 0.1020345687866211
Elapsed Time: 0.09938240051269531
Elapsed Time: 0.10870933532714844
Elapsed Time: 0.11127710342407227
Elapsed Time: 0.09742426872253418
Elapsed Time: 0.11077213287353516
Elapsed Time: 0.0973973274230957
Elapsed Time: 0.11187887191772461
Elapsed Time: 0.09761929512023926
Elapsed Time: 0.11094260215759277
Elapsed Time: 0.10239815711975098
Elapsed Time: 0.10508060455322266
Elapsed Time: 0.10274839401245117
Elapsed Time: 0.10653376579284668
Elapsed Time: 0.09583497047424316
Elapsed Time: 0.11146807670593262
Elapsed Time: 0.10035133361816406
Elapsed Time: 0.1100316047668457
Elapsed Time: 0.10203385353088379
Elapsed Time: 0.11593079566955566
Elapsed Time: 0.102

Elapsed Time: 0.11781859397888184
Elapsed Time: 0.10923504829406738
Elapsed Time: 0.11394739151000977
Elapsed Time: 0.10466265678405762
Elapsed Time: 0.12295198440551758
Elapsed Time: 0.11122584342956543
Elapsed Time: 0.1177370548248291
Elapsed Time: 0.10750842094421387
Elapsed Time: 0.11970186233520508
Elapsed Time: 0.10648298263549805
Elapsed Time: 0.12175679206848145
Elapsed Time: 0.1074531078338623
Elapsed Time: 0.12276220321655273
Elapsed Time: 0.10796356201171875
Elapsed Time: 0.1071481704711914
Elapsed Time: 0.1160881519317627
Elapsed Time: 0.11944723129272461
Elapsed Time: 0.11308765411376953
Elapsed Time: 0.11465692520141602
Elapsed Time: 0.10726690292358398
Elapsed Time: 0.1266770362854004
Elapsed Time: 0.10528850555419922
Elapsed Time: 0.11593747138977051
Elapsed Time: 0.10695123672485352
Elapsed Time: 0.11870384216308594
Elapsed Time: 0.10703921318054199
Elapsed Time: 0.11339521408081055
Elapsed Time: 0.10610032081604004
Elapsed Time: 0.12697339057922363
Elapsed Time: 0.110

Elapsed Time: 0.11660265922546387
Elapsed Time: 0.10783076286315918
Elapsed Time: 0.12274909019470215
Elapsed Time: 0.10923147201538086
Elapsed Time: 0.11117410659790039
Elapsed Time: 0.10496330261230469
Elapsed Time: 0.11558079719543457
Elapsed Time: 0.10910558700561523
Elapsed Time: 0.11858439445495605
Elapsed Time: 0.10559916496276855
Elapsed Time: 0.11762595176696777
Elapsed Time: 0.11103677749633789
Elapsed Time: 0.12018489837646484
Elapsed Time: 0.10843586921691895
Elapsed Time: 0.11423110961914062
Elapsed Time: 0.10781311988830566
Elapsed Time: 0.11529731750488281
Elapsed Time: 0.10936164855957031
Elapsed Time: 0.1213386058807373
Elapsed Time: 0.11029815673828125
Elapsed Time: 0.1165151596069336
Elapsed Time: 0.109649658203125
Elapsed Time: 0.11667394638061523
Elapsed Time: 0.11058998107910156
Elapsed Time: 0.11744880676269531
Elapsed Time: 0.10852670669555664
Elapsed Time: 0.11780261993408203
Elapsed Time: 0.11089134216308594
Elapsed Time: 0.1201021671295166
Elapsed Time: 0.105

Elapsed Time: 0.1196587085723877
Elapsed Time: 0.11012625694274902
Elapsed Time: 0.12506103515625
Elapsed Time: 0.10489511489868164
Elapsed Time: 0.11959958076477051
Elapsed Time: 0.10451698303222656
Elapsed Time: 0.11588168144226074
Elapsed Time: 0.10824131965637207
Elapsed Time: 0.11469841003417969
Elapsed Time: 0.10751891136169434
Elapsed Time: 0.11383819580078125
Elapsed Time: 0.11048293113708496
Elapsed Time: 0.11687493324279785
Elapsed Time: 0.1083834171295166
Elapsed Time: 0.11449337005615234
Elapsed Time: 0.10839581489562988
Elapsed Time: 0.11746811866760254
Elapsed Time: 0.10337281227111816
Elapsed Time: 0.11555838584899902
Elapsed Time: 0.10763931274414062
Elapsed Time: 0.1157081127166748
Elapsed Time: 0.10905933380126953
Elapsed Time: 0.11087512969970703
Elapsed Time: 0.10607433319091797
Elapsed Time: 0.11829876899719238
Elapsed Time: 0.10784077644348145
Elapsed Time: 0.11560988426208496
Elapsed Time: 0.11383414268493652
Elapsed Time: 0.1198420524597168
Elapsed Time: 0.10620

In [1]:
import numpy as np
import tensorflow as tf
import cv2
import time
import glob, os
from notify_run import Notify
notify = Notify()
from pydrive.drive import GoogleDrive
import serial #for Serial communication
import time   #for delay functions

delete_id=None 
print('Initializing Arduino')
arduino = serial.Serial(port='/dev/ttyACM0',baudrate = 9600)   #Create Serial port object called arduinoSerialData
print(arduino.readline())
print('Initialized Arduino')

#initializing Timing and mode

Mode=2
intruder=-2
    
# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
# org 
org = (0, 185) 
# fontScale 
fontScale = 1
# Red color in BGR 
color = (0, 0, 255) 
# Line thickness of 2 px 
thickness = 2
# Using cv2.putText() method 


def set_res(cap, x,y):
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, int(x))
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, int(y))
    return str(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),str(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.Session(graph=self.detection_graph)

        # Definite input and output Tensors for detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def processFrame(self, image):
        # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Actual detection.
        start_time = time.time()
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
        end_time = time.time()

        print("Elapsed Time:", end_time-start_time)

        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

    def close(self):
        self.sess.close()
        self.default_graph.close()
        
if __name__=='__main__':
    model_path = 'frozen_inference_graph.pb'
    odapi = DetectorAPI(path_to_ckpt=model_path)
    threshold = 0.4
    cap = cv2.VideoCapture(0)

    ## Saving recorded Video
    ## 

    while True:
        r, img = cap.read()
        if r==True:
            boxes, scores, classes, num = odapi.processFrame(img)
            counter=0
            # Visualization of the results of a detection.
            for i in range(len(boxes)):
                # Class 1 represents human
                if classes[i] == 1 and scores[i] > threshold:
                    box = boxes[i]
                    counter+=1
                    if intruder==0:
                        cv2.imwrite('screenshot.jpeg',img)
                        #NOTIFIER
                        file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
                        for file1 in file_list:
                            print('title: %s, id: %s' % (file1['title'], file1['id']))
                            if file1['title']=='screenshot.jpeg':
                                delete_id= file1['id']
                        if delete_id:
                            file_del = drive.CreateFile({'id':delete_id})
                            file_del.Delete()

                        file_drive = drive.CreateFile({'title': 'screenshot.jpeg' }) 
                        file_drive.SetContentFile("screenshot.jpeg") 
                        file_drive.Upload()
                        file_drive.InsertPermission({'type': 'anyone','value': 'anyone','role': 'reader'})
                        print("The file: " + 'screenshot.jpeg' + " has been uploaded")
                        print("All files have been uploaded")
                        file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
                        for file1 in file_list:
                            print('title: %s, id: %s' % (file1['title'], file1['id']))
                            if file1['title']=='screenshot.jpeg':
                                fetch_id= file1['id']
                        notify.send("Caution Intrusion Detected!!",'https://drive.google.com/open?id='+fetch_id)
                        
                        ##NOTIFICATION ENDS
                        
                    intruder-=1
            print("Humans Detected: "+str(counter))
            if(counter>0):
                if Mode==2:
                    arduino.write(str.encode('2'))
                    time.sleep(5)
                else:
                    arduino.write(str.encode('1'))
                    time.sleep(5)
            else:
                arduino.write(str.encode('0'))
            cv2.imshow("Video Footage", img)
            key = cv2.waitKey(1)
            if key &0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    cv2.destroyAllWindows()

Initializing Arduino
b'Arduino Strarted \n'
Initialized Arduino
Elapsed Time: 3.5613105297088623
Humans Detected: 1
Elapsed Time: 0.18284893035888672
Humans Detected: 1
Elapsed Time: 0.09715080261230469
Humans Detected: 1
Elapsed Time: 0.09132552146911621
Humans Detected: 1
Elapsed Time: 0.09313273429870605
Humans Detected: 1
